# This is my final Capstone Project notebook for the Data Science Certificate 

In [88]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Below you will find my code for the Segmenting and Clustering Neighborhoods in Toronto, Canada lab:

In [89]:
# import additional libraries needed for BeautifulSoup
from bs4 import BeautifulSoup
import requests

wiki_url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
req = requests.get(wiki_url)

soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [90]:
#remove 'Not assigned' rows from Borough column:
df = df[df.Borough != 'Not assigned']
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


In [91]:
#combine values with the same Postcode using groupby function:
df = df.groupby(['Postcode','Borough'],sort=False).agg(lambda x:','.join(x))
df 

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Not assigned
...,...,...
M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern


In [92]:
df = df.groupby(['Borough'])